In [45]:
import torch
import pandas as pd
import numpy as np

#("Bias",[1,1,1,1,1,1,1,1,1,1]),
# Into pandas dataframe
Farm=pd.DataFrame.from_items([("Bias",[1,1,1,1,1,1,1,1,1,1]),('Fertilizers',[6,10,12,14,16,18,22,24,26,32]),('Inseticides',[4,4,5,7,9,12,14,20,21,24])])
Farm_y=pd.DataFrame.from_items([("Corn",[40,44,46,48,52,58,60,68,74,80])])

'''Farm = (Farm - Farm.mean())/Farm.std()
Farm_y = (Farm_y - Farm_y.mean())/Farm_y.std()

# concatenate ones 
Farm = Farm.iloc[:,0:2]
ones = np.ones([Farm.shape[0],1])
Farm = np.concatenate((ones,Farm),axis=1)'''

# Numpy array
Farm_array=Farm.as_matrix()
Farm_y_array=Farm_y.as_matrix()
#print(Farm_array)

# Numpy array to tensor 
Farm_tensor=torch.from_numpy(Farm_array)
Farm_y_tensor=torch.from_numpy(Farm_y_array)
print(Farm_tensor)
print(Farm_y_tensor)

# Find Inverse 



    1     6     4
    1    10     4
    1    12     5
    1    14     7
    1    16     9
    1    18    12
    1    22    14
    1    24    20
    1    26    21
    1    32    24
[torch.LongTensor of size 10x3]


   40
   44
   46
   48
   52
   58
   60
   68
   74
   80
[torch.LongTensor of size 10x1]



In [46]:
# Now we need to find the pSeudo Inverse
#------------------------------------------------------------------------------
# Numpy transpose Find Transpose
Farm_array_transpose=np.transpose(Farm_array)
#print(Farm_array_transpose)
#------------------------------------------------------------------------------
tensor_transpose=torch.transpose(Farm_tensor,0,1)
print(tensor_transpose,Farm_tensor)
#Tranpose using torch


    1     1     1     1     1     1     1     1     1     1
    6    10    12    14    16    18    22    24    26    32
    4     4     5     7     9    12    14    20    21    24
[torch.LongTensor of size 3x10]
 
    1     6     4
    1    10     4
    1    12     5
    1    14     7
    1    16     9
    1    18    12
    1    22    14
    1    24    20
    1    26    21
    1    32    24
[torch.LongTensor of size 10x3]



In [47]:
# Calculate AT * A

TransOrig=torch.mm(tensor_transpose,Farm_tensor).double()
print(TransOrig)


   10   180   120
  180  3816  2684
  120  2684  1944
[torch.DoubleTensor of size 3x3]



In [48]:
# Take inverse 
TransOrig_Inverse=torch.inverse(TransOrig)
print(TransOrig_Inverse)

Original=torch.mm(TransOrig,TransOrig_Inverse)
#print(Original)


 1.3635 -0.1770  0.1602
-0.1770  0.0320 -0.0333
 0.1602 -0.0333  0.0366
[torch.DoubleTensor of size 3x3]



In [49]:
# Take the transOrig_inverse multiply with XT
tensor_transpose=tensor_transpose.double()
TransOrig_Inverse_Tran= torch.mm(TransOrig_Inverse,tensor_transpose)
print(TransOrig_Inverse_Tran)


 0.9423  0.2343  0.0405  0.0069 -0.0267  0.1000 -0.2876  0.3197  0.1259 -0.4554
-0.1180  0.0102  0.0409  0.0384  0.0359  0.0000  0.0615 -0.0743 -0.0435  0.0488
 0.1068 -0.0264 -0.0565 -0.0498 -0.0432  0.0000 -0.0600  0.0931  0.0631 -0.0270
[torch.DoubleTensor of size 3x10]



In [50]:
# Multiply this with y to get weight vector
Farm_y_tensor=Farm_y_tensor.double()
Weight=torch.mm(TransOrig_Inverse_Tran,Farm_y_tensor)

print(Weight)


 31.9807
  0.6501
  1.1099
[torch.DoubleTensor of size 3x1]



In [51]:
# Calculate y_prediction y_pred= w0 + w1X1 + w2+X2
print(Weight[0,0],Weight[1,0],Weight[2,0])


31.98067141403859 0.6500508646999523 1.1098677517802567


In [52]:
Farm_tensor



    1     6     4
    1    10     4
    1    12     5
    1    14     7
    1    16     9
    1    18    12
    1    22    14
    1    24    20
    1    26    21
    1    32    24
[torch.LongTensor of size 10x3]

In [53]:
x1=Farm_tensor[:,1:2].float()
x1


    6
   10
   12
   14
   16
   18
   22
   24
   26
   32
[torch.FloatTensor of size 10x1]

In [54]:
x2=Farm_tensor[:,2:3].float()
x2


    4
    4
    5
    7
    9
   12
   14
   20
   21
   24
[torch.FloatTensor of size 10x1]

In [55]:
W1=torch.randn(10,1)
W1=W1.fill_(Weight[1,0])
#W1=torch.transpose(W1,1,0)
W1


 0.6501
 0.6501
 0.6501
 0.6501
 0.6501
 0.6501
 0.6501
 0.6501
 0.6501
 0.6501
[torch.FloatTensor of size 10x1]

In [56]:
First_term=torch.randn(10,1)
First_term=First_term.fill_(0)
First_term=torch.addcmul(First_term,1,x1,W1)
First_term


  3.9003
  6.5005
  7.8006
  9.1007
 10.4008
 11.7009
 14.3011
 15.6012
 16.9013
 20.8016
[torch.FloatTensor of size 10x1]

In [57]:
W2=torch.randn(10,1)
W2=W1.fill_(Weight[2,0])



In [58]:
Second_term=torch.randn(10,1)
Second_term=Second_term.fill_(0)
Second_term=torch.addcmul(Second_term,1,x2,W2)
Second_term


  4.4395
  4.4395
  5.5493
  7.7691
  9.9888
 13.3184
 15.5381
 22.1974
 23.3072
 26.6368
[torch.FloatTensor of size 10x1]

In [59]:
W0=torch.randn(10,0)
W0=W0.fill_(Weight[0,0])
W0


 31.9807
 31.9807
 31.9807
 31.9807
 31.9807
 31.9807
 31.9807
 31.9807
 31.9807
 31.9807
[torch.FloatTensor of size 10]

In [60]:
# W0 + First_term + Second_term
yPred=W0+First_term+Second_term
print(yPred[:,0:1])
print(Farm_y_tensor)


 40.3204
 42.9207
 45.3306
 48.8505
 52.3703
 57.0000
 61.8199
 69.7793
 72.1892
 79.4191
[torch.FloatTensor of size 10x1]


   40
   44
   46
   48
   52
   58
   60
   68
   74
   80
[torch.DoubleTensor of size 10x1]



In [61]:
#print(yPred[:,0:1])
diff=Farm_y_tensor.float()-yPred[:,0:1].float()
print(diff)


-0.3204
 1.0793
 0.6694
-0.8505
-0.3703
 1.0000
-1.8199
-1.7793
 1.8108
 0.5809
[torch.FloatTensor of size 10x1]

